In [11]:
import os
import cv2
import numpy as np
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
import sys
sys.path.append('path/to/yolov5')  # Add the YOLOv5 directory to Python path


ImportError: cannot import name 'tarfile' from 'backports' (/opt/anaconda3/lib/python3.11/site-packages/backports/__init__.py)

In [3]:
#Scan through dataset 1 and remove images that are not bank card images

image_folder = 'APS360_Project_Dataset/dataset1/bank_card_images_train'
label_file = 'APS360_Project_Dataset/dataset1/bank_card_images_train_labels.csv'
# Load labels
labels_df = pd.read_csv(label_file)

# Filter out non-bank card images
labels_df = labels_df.dropna(subset=['card_num'])  # Keep only rows with a card number
labels_df['card_num'] = labels_df['card_num'].astype(str)  # Ensure labels are strings for sequence recognition

print(f"Total bank card images with card numbers: {len(labels_df)}")


Total bank card images with card numbers: 528


In [13]:
class CardNumberDataset(Dataset):
    def __init__(self, image_folder, labels_df, yolo_model, transform=None):
        self.image_folder = image_folder
        self.labels_df = labels_df
        self.yolo_model = yolo_model
        self.transform = transform

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        # Load image and label
        row = self.labels_df.iloc[idx]
        img_path = os.path.join(self.image_folder, f"{row['image_id']}.jpg")
        image = cv2.imread(img_path)
        label = row['card_number']

        # Use YOLO model to detect card number region
        results = self.yolo_model(image)  # Assuming the model accepts BGR images
        if len(results.xyxy[0]) > 0:  # Ensure there is at least one detection
            # Take the first detected bounding box
            x1, y1, x2, y2, conf, cls = results.xyxy[0][0]
            cropped_img = image[int(y1):int(y2), int(x1):int(x2)]
        else:
            print(f"No card number region detected for {img_path}")
            return None  # Skips this image if no detection found

        # Convert to grayscale, resize, and apply additional preprocessing
        cropped_img = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2GRAY)
        cropped_img = cv2.resize(cropped_img, (128, 32))  # Resize to standard input size for CRNN

        if self.transform:
            cropped_img = Image.fromarray(cropped_img)
            cropped_img = self.transform(cropped_img)

        return cropped_img, label


In [14]:
from torchvision import transforms

# Define image preprocessing
preprocess_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize grayscale image
])

yolo_model = torch.hub.load('ultralytics/yolov5', 'custom', path='card_number_detection/yolov5/runs/train/card_number_detector6/weights/best.pt')

# Create dataset and dataloader
card_dataset = CardNumberDataset(image_folder, labels_df, yolo_model, transform=preprocess_transform)
card_dataloader = DataLoader(card_dataset, batch_size=16, shuffle=True, collate_fn=lambda x: [i for i in x if i])

# Check data loading
for images, labels in card_dataloader:
    print("Batch of images:", images.size())
    print("Batch of labels:", labels)
    break


Using cache found in /Users/johnson/.cache/torch/hub/ultralytics_yolov5_master


ImportError: cannot import name 'tarfile' from 'backports' (/opt/anaconda3/lib/python3.11/site-packages/backports/__init__.py)